# 車を検出し、その周囲に境界ボックスを配置します

このノートブックでは、Yolo モデルによって検出された車の周囲にボックスを描画する方法を学習します。

In [ ]:
# このラボ用に設計されたワークベンチ イメージを使用しなかった場合は、コメントを解除して次の行を実行して、必要なパッケージをインストールできます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

from ultralytics import YOLO
from PIL import Image

In [ ]:
# 物体検出には YOLOv8m モデルを使用します。

model = YOLO("yolov8m.pt")

In [ ]:
# テスト画像に対するモデル予測の結果を取得します。

img = "images/carImage0.jpg"
results = model.predict(img)

In [ ]:
# YOLO を使用すると、1 つだけでなく一連の画像を送信でき、一連の結果が得られます。
# 送信した画像は 1 つだけなので、結果の配列から最初 (唯一) の要素のみを取得する必要があります。

result = results[0]

In [ ]:
# 見つかったボックスの数を検出します。

len(result.boxes)

In [ ]:
# ボックスを分析します。

box = result.boxes[0]
print("Object type:", box.cls)
print("Coordinates:", box.xyxy)
print("Probability:", box.conf)

In [ ]:
# Tensor から実際の値をアンパックする

cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
conf = box.conf[0].item()
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

In [ ]:
# COCO は、YOLO モデルがトレーニングされたデータセットです。検出するように訓練されたオブジェクトはクラスに編成されます。これは、 "Object type" フィールドで取得した情報です。
# YOLOv8 結果オブジェクトには、これらのクラスの 'names' プロパティも含まれています。

print(result.names)


クラス番号 '2' が 'car' オブジェクトに対応していることがわかります。  したがって、結果の境界ボックスは、検出された 'car' に対応します。
画像の上にボックスを描いてみましょう！


In [ ]:
# まず、座標をリストに入れ、四捨五入します。
# 次に、result.names ディクショナリを使用して、検出されたオブジェクト クラスの名前を ID によって取得します。

cords = box.xyxy[0].tolist()
cords = [round(x) for x in cords]
class_id = result.names[box.cls[0].item()]
conf = round(box.conf[0].item(), 2)
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

In [ ]:
# すべてのボックスをループして情報を抽出しましょう。

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

In [ ]:
# 画像上に、ボックス、クラス名、確率（モデルが検出についてどの程度確信しているか） を描画します。

Image.fromarray(result.plot()[:, :, ::-1])

さて、前回のノートブックでテストした複数の車の画像（carImage4.jpg）に戻り、yolo が実際に画像内のすべての 'cars' を識別できるかどうかを確認してみましょう。

In [ ]:
# これは前のセルで使用したのと同じコードですが、同じコードで問題ありません。

results = model.predict("images/carImage4.jpg")

result = results[0]

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

Image.fromarray(result.plot()[:,:,::-1])

YOLO モデルでは、画像の '一番後ろ' にあるいくつかの車を見逃していることがわかります。  しかし全体として、モデルはこの画像内の複数の車を識別するという点で優れた仕事をしました。  そしてさらに重要なのは、識別された車が '境界ボックス' で囲まれていることがわかります。!

完了したら、ノートブックを閉じて次のページに進むことができます。

yolo モデルによって認識された車の周囲に境界ボックスを配置できるようになったので、車の '衝突' を識別するために YOLO モデルを再トレーニングしましょう。